# Preprocessing danych

Zacznijmy do zaimportowania podstawowych modułów i bibliotek. Upewnij się, że masz zainstalowany ```scikit-learn```.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
assert sklearn.__version__ >= "0.20"

Następnie, pobieramy zbiór ze słynnego konkursu "Titanic - Machine Learning from Disaster" z poniższego linku. <br/>
https://www.kaggle.com/c/titanic/data

Kaggle to ważna strona w świecie data science i machine learning'u. Można na niej znaleźć masę zbiorów danych, praktyczne mikrokursy, notebooki i to z czego słynie najbardziej - competitions(w tym przyszłe lokalne BIT AI ;) ). Jeśli jeszcze tego nie zrobiłeś/aś, gorąco zachęcam do założenia konta.

Dane wypakowujemy do wybranego folderu, a następnie wczytujemy je do data frame'ów. Poniższy kod zakłada, że pliki są w tym samym miejscu co ten notebook.

In [ ]:
datapath = '.' #change accordingly

def load_data(filename):
    csv_path = os.path.join(datapath, filename)
    return pd.read_csv(csv_path)

In [ ]:
%%time
train_data = load_data('train.csv')
test_data = load_data('test.csv')

In [ ]:
?pd.read_csv

In [ ]:
??pd.read_csv

## Podstawowa analiza danych

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
%matplotlib notebook
train_data.hist(figsize=(8,8))
plt.show()

In [ ]:
num_train_data = train_data.select_dtypes(exclude=['object'])
cat_train_X = train_data.select_dtypes(include=['object'])

In [ ]:
num_train_X, y = num_train_data.drop('Survived', axis=1), num_train_data['Survived']

## Eliminacja "nieużytecznych" zmiennych

In [ ]:
corr_matrix = train_data.corr()
corr_matrix['Survived'].sort_values(ascending=False)

In [ ]:
num_train_X = num_train_X.drop('PassengerId', axis=1)
cat_train_X = cat_train_X.drop('Name', axis=1)

## Problem brakujących wartości

In [ ]:
cat_train_X = cat_train_X.drop('Cabin', axis=1)

In [ ]:
from sklearn.impute import SimpleImputer

num_imputer = SimpleImputer(strategy='median')
num_imputer.fit(num_train_X)
imputed_num_train_X = num_imputer.transform(num_train_X)

In [ ]:
num_train_X = pd.DataFrame(imputed_num_train_X,
                              columns=num_train_X.columns,
                              index=num_train_X.index)
num_train_X.info()

In [ ]:
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_imputer.fit(cat_train_X)
imputed_cat_train_X = cat_imputer.transform(cat_train_X)

In [ ]:
cat_train_X = pd.DataFrame(imputed_cat_train_X,
                              columns=cat_train_X.columns,
                              index=cat_train_X.index)
cat_train_X.info()

---
<h2><span style="color:orange">Bonus</span></h2>
Użyliśmy tradycyjnej imputacji. Poniżej są wykresy przedstawiające obserwacje, które nie zawierają wieku. Widzisz jakieś zależności? Spróbuj dokonać imputacji wielowymiarowej. Poprawia wynik naszego modelu? <br/>

*Tip* `IterativeImputer`

In [ ]:
nan_age_df = train_data[train_data['Age'].isna()]
nan_age_df.hist(figsize=(8,8))
plt.show()

---

## Zmienne kategoryczne

In [ ]:
cat_train_X['Sex'].value_counts()

In [ ]:
cat_train_X['Embarked'].value_counts()

In [ ]:
cat_train_X['Ticket'].value_counts()

In [ ]:
cat_train_X = cat_train_X.drop('Ticket', axis=1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoder.fit(cat_train_X)
encoder.categories_

In [ ]:
cat_train_X = encoder.transform(cat_train_X)
cat_train_X = pd.DataFrame(cat_train_X,
                              columns=['Female', 'Male', 'C', 'Q', 'S'])
cat_train_X

---
<h2><span style="color:orange">Bonus</span></h2>


Pominęliśmy być może istotną zmienną `Ticket`. Spróbuj ją zakodować wykorzystując hashing lub kodowanie binarne.
Duży plus jeżeli zrobisz to samodzielnie, ale możesz wykorzystać bibliotekę http://contrib.scikit-learn.org/category_encoders/.
---

## Skalowanie

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(num_train_X)
scaled_num_train_X = scaler.transform(num_train_X)
num_train_X = pd.DataFrame(scaled_num_train_X,
                           columns=num_train_X.columns)

In [ ]:
num_train_X.describe()

## Dyskretyzacja

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
?pd.cut

## Modelowanie

In [ ]:
train_X = pd.concat([num_train_X, cat_train_X], axis=1)
train_X.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()
classifier.fit(train_X, y)

In [ ]:
from sklearn.model_selection import cross_val_score

train_pred = classifier.predict(train_X)
train_scores = cross_val_score(classifier, train_X, y,
                               scoring='accuracy', cv=10)
np.mean(train_scores)

## Preprocessing danych testowych

In [ ]:
def preprocess(df, num_imputer, cat_imputer, one_hot_encoder,
               scaler):
    num_df = df.select_dtypes(exclude=['object'])
    cat_df = df.select_dtypes(include=['object'])
    #redundancy removal
    num_df = num_df.drop('PassengerId', axis=1)
    cat_df = cat_df.drop(['Name', 'Cabin'], axis=1)
    #handle missing values
    imputed_num = num_imputer.transform(num_df) #notice that we do NOT fit
    imputed_cat = cat_imputer.transform(cat_df)
    num_df = pd.DataFrame(imputed_num,
                          columns=num_df.columns,
                          index=num_df.index)
    cat_df = pd.DataFrame(imputed_cat,
                          columns=cat_df.columns,
                          index=cat_df.index)
    cat_df = cat_df.drop('Ticket', axis=1)
    #encode categorical variables
    cat_df = encoder.transform(cat_df)
    cat_df = pd.DataFrame(cat_df,
                          columns=['Female', 'Male', 'C', 'Q', 'S'])
    #scaling
    scaled_num = scaler.transform(num_df)
    num_df = pd.DataFrame(scaled_num,
                          columns=num_df.columns)
    result_df = pd.concat([num_df, cat_df], axis=1)
    return result_df

In [ ]:
test_X = preprocess(test_data, num_imputer, cat_imputer, encoder, scaler)
test_X.info()

In [ ]:
test_pred = classifier.predict(test_X)

In [ ]:
ids = np.array([len(train_X) + (i+1) for i in range(len(test_pred))], dtype=int)
ids = ids.reshape(-1, 1)
test_pred = test_pred.reshape(-1, 1)
pred_df = pd.DataFrame(np.concatenate((ids, test_pred), axis=1),
                       columns=['PassengerId', 'Survived'])

In [ ]:
pred_df

In [ ]:
pred_df.to_csv('titanic_predictions.csv', index=False)

Gratulacje! Stworzyliśmy pełnoprawny model machine learningu.

---
<h2><span style="color:orange">Bonus</span></h2>

Stworzyliśmy model, ale wykorzystaliśmy domyślne(bardzo przemyślane) hiperparametry, aby go ulepszyć, musimy znaleźć odpowiednie wartości dla naszego problemu.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
     'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 10, 12, 15],
     'weights': ['uniform', 'distance'],
     'p': [1, 2, 3]}
  ]

In [ ]:
%%time
grid_search = GridSearchCV(classifier, param_grid, cv=5,
                           scoring='accuracy',
                           return_train_score=True)
grid_search.fit(train_X, y)
grid_search.best_params_

---

<h2><span style="color:orange">Bonus II</span></h2>

Preprocessing może być żmudnym procesem. To w jaki sposób przetworzyliśmy dane treningowe, musimy powtórzyć dla danych testowych. Tworzenie dużych i długich funkcji, tak jak `preprocess` może być niewygodne i niesie za sobą ograniczenia.
Ponadto, zwróć uwagę, że etapy preprocessingu, również można(nawet trzeba) tuningować poprzez dobór odpowiednich hiperparametrów. W obecnej formie jest to mocno utrudnione. 
Z pomocą przychodzą pipeline'y:


https://blog.prokulski.science/2020/10/10/pipeline-w-scikit-learn/

Zapoznaj się z artykułem i spróbuj zbudować prosty pipeline dla danych numerycznych.